In [1]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH "] = "true"

import h5py
from tqdm.notebook import tqdm

from deepalign import Dataset
from deepalign import fs
from deepalign.alignments import ALIGNERS
from deepalign.alignments.confnet import ConfNet

To speed up the evaluation, we are caching all results. You will have received these cache files with the download of the GitHub release. In case you want to run your own experiments, this is the code.

In [2]:
def get_aligner(model_file, dataset):
    if 'confnet' in model_file.ad:
        aligner = ALIGNERS[model_file.ad[:-2]](dataset,
                                               use_case_attributes=model_file.use_case_attributes,
                                               use_event_attributes=model_file.use_event_attributes,
                                               align=True)
        aligner.load(str(fs.MODEL_DIR / model_file.name), dataset)
    else:
        aligner = ALIGNERS[model_file.ad]()
        aligner.load(str(fs.MODEL_DIR / model_file.name))
    
    return aligner

In [3]:
synthetic = ['paper', 'p2p', 'small', 'medium', 'huge', 'wide']

models = sorted(list(set([f.name.replace('_forward', '').replace('_backward', '')
                          for f in fs.get_aligner_files()])))

models = [m for m in models if not (fs.RESULT_DIR / (fs.ModelFile(m).name + '.h5')).exists()]

for model in tqdm(models):
    model_file = fs.AlignerFile(model)
    dataset = Dataset(model_file.event_log_name,
                      use_case_attributes=model_file.use_case_attributes,
                      use_event_attributes=model_file.use_event_attributes)
    aligner = get_aligner(model_file, dataset)

    if isinstance(aligner, ConfNet):
        alignments, beams, costs = aligner.batch_align(dataset, batch_size=5000)
    else:
        try:
            alignments, beams, costs = aligner.align(dataset)
        except Exception as e:
            print(e)
            continue

    with h5py.File(str(fs.RESULT_DIR / (model_file.name + '.h5')), 'w') as file:
        file.create_dataset('alignments', data=alignments, compression="gzip", compression_opts=9)
        file.create_dataset('beams', data=beams, compression="gzip", compression_opts=9)
        file.create_dataset('costs', data=costs, compression="gzip", compression_opts=9)

Step 1 → 1.389997959136963s (25000, 27) finished=3327
Step 2 ← 1.5860018730163574s (25000, 27) finished=3327
Step 3 → 1.537001132965088s (25000, 27) finished=3704
Step 4 ← 1.1989994049072266s (25000, 27) finished=4433
Step 5 → 0.5530028343200684s (25000, 27) finished=4859
Step 6 ← 0.17499423027038574s (25000, 27) finished=4940
Step 7 → 0.08500385284423828s (25000, 27) finished=4986
Step 8 ← 0.044996023178100586s (25000, 27) finished=4989
Step 9 → 0.041002750396728516s (25000, 27) finished=4999
Step 10 ← 0.03700065612792969s (25000, 27) finished=5000
Step 1 → 2.1740033626556396s (25000, 27) finished=3650
Step 2 ← 1.8119983673095703s (25000, 27) finished=3650
Step 3 → 1.7429990768432617s (25000, 27) finished=3789
Step 4 ← 1.5459997653961182s (25000, 27) finished=4298
Step 5 → 0.9430015087127686s (25000, 27) finished=4703
Step 6 ← 0.3959999084472656s (25000, 27) finished=4756
Step 7 → 0.35599851608276367s (25000, 27) finished=4789
Step 8 ← 0.32599902153015137s (25000, 27) finished=4799
St

Y:\Repos_Uni\deepalign-master\deepalign\processmining\log.py:58: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(self.cases)[indices]
wide-0.3-1: 100%|██████████| 571/571 [00:08<00:00, 66.41it/s]
